In [1]:
!pip install deeptables[gpu]
from google.colab import drive
from google.colab import files
!pip install wandb
import wandb
wandb.init()

drive.mount('/content/gdrive')

wandb: Currently logged in as: thunderock (use `wandb login --relogin` to force relogin)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from IPython.display import FileLink
# from tqdm import tqdm_notebook 
from tqdm import tqdm
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import PredefinedSplit
import warnings
from joblib import Parallel, delayed
# from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
# from tsfresh.utilities.dataframe_functions import impute
from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.datasets import dsutils
warnings.filterwarnings('ignore')


In [3]:
train_file = '/content/gdrive/My Drive/training_files/result_{}.csv'
super_train = '/content/gdrive/My Drive/super_train.csv'


In [4]:
stock = 1


In [5]:
tdf = pd.read_csv(super_train, index_col='ID')

tdf

,Open_hat,Open_trend,Open_lower,Open_upper,Open_trend_lower,Open_ trend_upper,High_hat,High_trend,High_lower,High_upper,High_trend_lower,High_ trend_upper,Low_hat,Low_trend,Low_lower,Low_upper,Low_trend_lower,Low_ trend_upper,Close_hat,Close_trend,Close_lower,Close_upper,Close_trend_lower,Close_ trend_upper,stock,Date,Open,High,Low,Close,holiday,unpredictability_score,Open_diff,Low_diff,High_diff,Close_diff
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_0,81.025967,82.386498,78.803918,83.056482,82.386498,82.386498,81.202899,82.597286,79.340480,83.157889,82.597286,82.597286,80.911921,82.566861,78.592072,83.164115,82.566861,82.566861,80.834147,82.409348,78.733809,82.942531,82.409348,82.409348,0,2017-01-03,82.9961,82.7396,82.9144,82.8101,1,7,1.970133,2.002479,1.536701,1.975953
id_1,80.865875,82.437997,78.822365,82.974286,82.437997,82.437997,81.046504,82.643819,79.061205,83.051594,82.643819,82.643819,80.911873,82.618169,78.718842,82.964155,82.618169,82.618169,80.817877,82.460411,78.649237,82.960737,82.460411,82.460411,0,2017-01-04,83.1312,83.1669,83.3779,82.9690,0,7,2.265325,2.466027,2.120396,2.151123
id_2,80.976781,82.489497,78.898351,83.082205,82.489497,82.489497,81.080195,82.690352,79.154399,83.025130,82.690352,82.690352,81.094516,82.669477,79.022828,83.123835,82.669477,82.669477,80.945351,82.511474,78.570194,83.129957,82.511474,82.511474,0,2017-01-05,82.6622,82.7634,82.8984,82.8578,0,7,1.685419,1.803884,1.683205,1.912449
id_3,81.060195,82.540996,78.763372,83.062250,82.540996,82.540996,81.148817,82.736884,79.077222,83.003408,82.736884,82.736884,81.213930,82.720784,79.160796,83.262724,82.720784,82.720784,81.086557,82.562536,78.789351,83.149835,82.562536,82.562536,0,2017-01-06,83.0279,82.7950,82.8425,82.7385,0,7,1.967705,1.628570,1.646183,1.651943
id_4,81.141042,82.695494,79.101468,83.174879,82.695494,82.695494,81.171321,82.876482,78.954744,83.127788,82.876482,82.876482,81.413543,82.874707,79.312939,83.596425,82.874707,82.874707,81.191365,82.715724,79.242205,83.400230,82.715724,82.715724,0,2017-01-09,82.3761,82.0828,82.1473,81.8641,0,7,1.235058,0.733757,0.911479,0.672735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_77657,133.246558,130.185331,130.398763,135.963138,128.413669,131.840591,132.715079,129.880891,129.605239,135.501505,128.091986,131.771849,133.725970,130.476072,130.849425,136.848623,128.698410,132.143296,133.109925,129.879023,129.995531,135.960626,128.304842,131.653423,102,2019-12-24,NaN,NaN,NaN,NaN,0,6,0.153895,0.051313,0.258674,0.197796
id_77658,133.613717,130.157991,130.816426,136.724601,128.277605,131.948423,133.031871,129.850858,129.984751,135.854295,127.919271,131.863860,133.743532,130.447991,130.782226,136.626157,128.560217,132.196857,133.159597,129.844759,130.238639,135.972599,128.176349,131.751620,102,2019-12-26,NaN,NaN,NaN,NaN,1,6,0.457144,0.291064,0.598355,0.414344
id_77659,133.402365,130.144321,130.475015,136.215723,128.193173,132.004618,132.892882,129.835841,129.924311,135.739193,127.854733,131.879656,133.900849,130.433950,130.789703,136.960450,128.506409,132.251117,133.339552,129.827627,130.587664,136.287095,128.089915,131.775392,102,2019-12-27,NaN,NaN,NaN,NaN,0,6,0.152253,0.143879,0.246128,0.265842


In [6]:
cat_cols = [ 
    'holiday',
    'stock',
    'day',
     'month',
     'year',
     'dayofweek',
     'dayofyear',
     'weekofyear',
    'unpredictability_score']
excluded_cols = ['Close_hat', 'Open_hat', 'High_hat', 'Low_hat']
                 

In [7]:
def expand_df(dframe):
    dFrame = dframe.copy()
    dFrame['day'] = dFrame.Date.apply(lambda x: x.day)
    dFrame['month'] = dFrame.Date.apply(lambda x: x.month)
    dFrame['year'] = dFrame.Date.apply(lambda x: x.year)
    dFrame['dayofweek'] = dFrame.Date.apply(lambda x: x.dayofweek)
    dFrame['dayofyear'] = dFrame.Date.apply(lambda x: x.dayofyear)
    dFrame['weekofyear'] = dFrame.Date.apply(lambda x: x.weekofyear)
    dFrame['year_diff'] = dFrame.Date.apply(lambda x: x.year - 2017)
    dFrame['days_so_far_skipped'] = dFrame.Date.apply(lambda x: dFrame[dFrame.Date < x].shape[0])
    dFrame['days_so_far'] = dFrame.Date.apply(lambda x: (x - pd.Timestamp('2017-01-03')).days)
    return dFrame
#     return dFrame[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat'] + ['Date'] + cat_cols + ['year_diff', 'days_so_far_skipped', 'days_so_far'] + ['Close', 'Open', 'High', 'Low'] ]

In [8]:
def get_rolling_mean(dframe, col, idx, days=30, met='mean'):
#     print(col, idx, days, met)
    if met == 'mean':
        return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].mean()
    if met == 'max':
        return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].max()
    return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].min()

def rolled_mean(dframe, timeshift=30):
    dframe['ID'] = dframe.index

    for col in excluded_cols:
        dframe[col + '_roll_mean_per_mon'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=timeshift, met='mean'))
        dframe[col + '_roll_max_per_mon'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=timeshift, met='max'))
        dframe[col + '_roll_min_per_mon'] = dframe['ID'].apply(lambda x:get_rolling_mean(dframe, col, x, days=timeshift, met='min'))
        dframe[col + '_roll_range_per_mon'] = dframe[col + '_roll_max_per_mon'] - dframe[col + '_roll_min_per_mon']
        dframe[col + '_roll_mean_per_d'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=2, met='mean'))
        dframe[col + '_roll_max_per_d'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=2, met='max'))
        dframe[col + '_roll_min_per_d'] = dframe['ID'].apply(lambda x:get_rolling_mean(dframe, col, x, days=2, met='min'))
        dframe[col + '_roll_range_per_d'] = dframe[col + '_roll_max_per_d'] - dframe[col + '_roll_min_per_d']
    
    return dframe.set_index('ID', drop=True)

In [9]:
def split_time_series(ts, ys, train_size):
    
    split_ind = int(train_size*ts.shape[0])
    train_ts, val_ts, train_ys, val_ys = ts[:split_ind], ts[split_ind:], ys[:split_ind], ys[split_ind:]
    return (train_ts, val_ts, train_ys, val_ys)

In [10]:
df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
df = expand_df(df)

df = rolled_mean(df)

encoder = LabelEncoder()
for col in cat_cols:
    df[col] = encoder.fit_transform(df[col])

df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

X_tr, X_val, y_tr, y_val = split_time_series(X, y['Close'], train_size=.8)


In [11]:
df.columns.tolist()

['stock',
 'unpredictability_score',
 'Close',
 'High',
 'Low',
 'Open',
 'holiday',
 'Date',
 'Open_upper__benford_correlation',
 'Open_hat__benford_correlation',
 'Low_hat__benford_correlation',
 'Low_trend__benford_correlation',
 'Low_trend_lower__benford_correlation',
 'Low__trend_upper__benford_correlation',
 'Close_upper__benford_correlation',
 'Open_hat__fft_coefficient__attr__abs___coeff_0',
 'Open_hat__fft_coefficient__attr__real___coeff_0',
 'Open_hat__cwt_coefficients__coeff_0__w_20__widths__2__5__10__20_',
 'Open_hat__cwt_coefficients__coeff_0__w_10__widths__2__5__10__20_',
 'Open_hat__cwt_coefficients__coeff_0__w_5__widths__2__5__10__20_',
 'Open_hat__cwt_coefficients__coeff_0__w_2__widths__2__5__10__20_',
 'Open_hat__sum_values',
 'Open_hat__quantile__q_0_8',
 'Open_hat__abs_energy',
 'Open_hat__median',
 'Open_hat__mean',
 'Open_hat__quantile__q_0_9',
 'Open_hat__minimum',
 'Open_hat__maximum',
 'Open_hat__quantile__q_0_2',
 'Open_hat__quantile__q_0_3',
 'Open_hat__quant

In [12]:
X

,stock,unpredictability_score,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,Low_hat__benford_correlation,Low_trend__benford_correlation,Low_trend_lower__benford_correlation,Low__trend_upper__benford_correlation,Close_upper__benford_correlation,Open_hat__fft_coefficient__attr__abs___coeff_0,Open_hat__fft_coefficient__attr__real___coeff_0,Open_hat__cwt_coefficients__coeff_0__w_20__widths__2__5__10__20_,Open_hat__cwt_coefficients__coeff_0__w_10__widths__2__5__10__20_,Open_hat__cwt_coefficients__coeff_0__w_5__widths__2__5__10__20_,Open_hat__cwt_coefficients__coeff_0__w_2__widths__2__5__10__20_,Open_hat__sum_values,Open_hat__quantile__q_0_8,Open_hat__abs_energy,Open_hat__median,Open_hat__mean,Open_hat__quantile__q_0_9,Open_hat__minimum,Open_hat__maximum,Open_hat__quantile__q_0_2,Open_hat__quantile__q_0_3,Open_hat__quantile__q_0_4,Open_hat__quantile__q_0_6,Open_hat__quantile__q_0_7,Open_hat__quantile__q_0_1,Open_lower__sum_values,Open_lower__abs_energy,Open_lower__median,Open_lower__mean,Open_lower__quantile__q_0_3,Open_lower__minimum,Open_lower__quantile__q_0_1,Open_lower__quantile__q_0_2,Open_lower__quantile__q_0_7,...,month,year,dayofweek,dayofyear,weekofyear,year_diff,days_so_far_skipped,days_so_far,Close_hat_roll_mean_per_mon,Close_hat_roll_max_per_mon,Close_hat_roll_min_per_mon,Close_hat_roll_range_per_mon,Close_hat_roll_mean_per_d,Close_hat_roll_max_per_d,Close_hat_roll_min_per_d,Close_hat_roll_range_per_d,Open_hat_roll_mean_per_mon,Open_hat_roll_max_per_mon,Open_hat_roll_min_per_mon,Open_hat_roll_range_per_mon,Open_hat_roll_mean_per_d,Open_hat_roll_max_per_d,Open_hat_roll_min_per_d,Open_hat_roll_range_per_d,High_hat_roll_mean_per_mon,High_hat_roll_max_per_mon,High_hat_roll_min_per_mon,High_hat_roll_range_per_mon,High_hat_roll_mean_per_d,High_hat_roll_max_per_d,High_hat_roll_min_per_d,High_hat_roll_range_per_d,Low_hat_roll_mean_per_mon,Low_hat_roll_max_per_mon,Low_hat_roll_min_per_mon,Low_hat_roll_range_per_mon,Low_hat_roll_mean_per_d,Low_hat_roll_max_per_d,Low_hat_roll_min_per_d,Low_hat_roll_range_per_d
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_754,0,0,1,2017-01-03,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,38.212551,38.212551,7.410934,10.480643,14.821868,23.435431,38.212551,38.212551,1460.199079,38.212551,38.212551,38.212551,38.212551,38.212551,38.212551,38.212551,38.212551,38.212551,38.212551,38.212551,37.959566,1440.928641,37.959566,37.959566,37.959566,37.959566,37.959566,37.959566,37.959566,...,0,0,1,1,0,0,0,0,38.598828,38.821051,38.243238,0.577813,38.271107,38.298977,38.243238,0.055739,38.554120,38.769382,38.212551,0.556831,38.237210,38.261870,38.212551,0.049318,38.373847,38.597446,38.040274,0.557173,38.059897,38.079520,38.040274,0.039247,38.824593,39.048402,38.501096,0.547305,38.528666,38.556235,38.501096,0.055138
id_755,0,0,0,2017-01-04,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,38.261870,38.261870,7.420499,10.494170,14.840997,23.465677,38.261870,38.261870,1463.970659,38.261870,38.261870,38.261870,38.261870,38.261870,38.261870,38.261870,38.261870,38.261870,38.261870,38.261870,37.989241,1443.182427,37.989241,37.989241,37.989241,37.989241,37.989241,37.989241,37.989241,...,0,0,2,2,0,0,1,1,38.609333,38.821051,38.298977,0.522074,38.316837,38.334698,38.298977,0.035722,38.563685,38.769382,38.261870,0.507512,38.280330,38.298790,38.261870,0.036921,38.383342,38.597446,38.079520,0.517926,38.096333,38.113146,38.079520,0.033626,38.833514,39.048402,38.556235,0.492167,38.579376,38.602518,38.556235,0.046283
id_756,0,0,0,2017-01-05,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,38.298790,38.298790,7.427659,10.504296,14.855318,23.488320,38.298790,38.298790,1466.797353,38.298790,38.298790,38.298790,38.298790,38.298790,38.298790,38.298790,38.298790,38.298790,38.298790,38.298790,38.061646,1448.688890,38.061646,38.061646,38.061646,38.061646,38.061646,38.061646,38.061646,...,0,0,3,3,0,0,2,2,38.618231,38.821051,38

In [13]:
def rmse(y_true, y_pred):
    return abs(mean_squared_error(y_true, y_pred, squared=False))

In [14]:
conf = ModelConfig(
    metrics=['RootMeanSquaredError'], 
    nets=['dnn_nets'],
    dnn_params={
        'hidden_units': ((256, 0.3, True), (256, 0.3, True)),
        'dnn_activation': 'relu',
    },
    earlystopping_patience=5,
)
preds1 = []

def get_predictions(stock):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
    df = expand_df(df)

    df = rolled_mean(df)

    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])

    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close']][df['Close'].isna()]

    X_tr, X_val, y_tr, y_val = split_time_series(X, y, train_size=.8)
    conf = ModelConfig(
        metrics=['RootMeanSquaredError'], 
        nets=['dnn_nets'],
        categorical_columns=cat_cols, 
        dnn_params={
            'hidden_units': ((256, 0.3, True), (256, 0.3, True), (256, 0.3, True)),
            'dnn_activation': 'relu',
        },
        earlystopping_patience=5,
    )
    dt = DeepTable(config=conf)
    dt.fit(X, y.pop('Close'), epochs=500)
    return pd.DataFrame({'Score':dt.evaluate(X_val, y_val.pop('Close'))['root_mean_squared_error'], 'ID': X_test.index, 'Close': dt.predict(X_test)[:, 0]})


num_cores = 5
preds1 = Parallel(n_jobs=num_cores)(delayed(get_predictions)(stock) for stock in tqdm(range(103)))
# preds1 = [get_predictions(stock) for stock in tqdm(range(1))]

100%|██████████| 103/103 [1:29:53<00:00, 52.37s/it]


In [15]:

pd.concat(preds1)[['ID', 'Close']].to_csv('result_dt.csv', index=False)

pd.concat(preds1).to_csv('result_dt_analysis.csv', index=False)



In [16]:
pd.concat(preds1)

,Score,ID,Close
0,2.911390,id_713,114.789406
1,2.911390,id_714,113.715645
2,2.911390,id_715,112.088173
3,2.911390,id_716,111.690582
4,2.911390,id_717,112.766846
...,...,...,...
36,1.558009,id_77657,131.845673
37,1.558009,id_77658,130.695831
38,1.558009,id_77659,130.980240
39,1.558009,id_77660,131.521027


In [17]:
files.download('result_dt.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
files.download('result_dt_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:

!cat result_dt.csv

ID,Close
id_713,114.789406
id_714,113.715645
id_715,112.08817
id_716,111.69058
id_717,112.766846
id_718,113.93988
id_719,114.00936
id_720,113.16761
id_721,113.9827
id_722,114.18333
id_723,115.1845
id_724,115.494865
id_725,114.9074
id_726,116.115
id_727,115.89148
id_728,116.58039
id_729,117.1613
id_730,116.73551
id_731,117.21541
id_732,117.913895
id_733,117.9825
id_734,117.31372
id_735,117.8853
id_736,117.68355
id_737,118.13087
id_738,118.41887
id_739,118.597565
id_740,118.37547
id_741,118.276085
id_742,118.411
id_743,119.174355
id_744,119.31051
id_745,118.52105
id_746,118.2842
id_747,118.71601
id_748,118.79797
id_749,118.130135
id_750,117.982216
id_751,118.23829
id_752,119.22345
id_753,118.72185
id_1467,44.872135
id_1468,44.850487
id_1469,44.883453
id_1470,44.781784
id_1471,44.809547
id_1472,44.723587
id_1473,44.69427
id_1474,44.762615
id_1475,44.677914
id_1476,44.671154
id_1477,44.675056
id_1478,44.61884
id_1479,44.707844
id_1480,44.65598
id_1481,44.647125
id_1482,44.630577
id_1483,44

In [20]:
!cat result_dt_analysis.csv

Score,ID,Close
2.9113900661468506,id_713,114.789406
2.9113900661468506,id_714,113.715645
2.9113900661468506,id_715,112.08817
2.9113900661468506,id_716,111.69058
2.9113900661468506,id_717,112.766846
2.9113900661468506,id_718,113.93988
2.9113900661468506,id_719,114.00936
2.9113900661468506,id_720,113.16761
2.9113900661468506,id_721,113.9827
2.9113900661468506,id_722,114.18333
2.9113900661468506,id_723,115.1845
2.9113900661468506,id_724,115.494865
2.9113900661468506,id_725,114.9074
2.9113900661468506,id_726,116.115
2.9113900661468506,id_727,115.89148
2.9113900661468506,id_728,116.58039
2.9113900661468506,id_729,117.1613
2.9113900661468506,id_730,116.73551
2.9113900661468506,id_731,117.21541
2.9113900661468506,id_732,117.913895
2.9113900661468506,id_733,117.9825
2.9113900661468506,id_734,117.31372
2.9113900661468506,id_735,117.8853
2.9113900661468506,id_736,117.68355
2.9113900661468506,id_737,118.13087
2.9113900661468506,id_738,118.41887
2.9113900661468506,id_739,118.597565
2.9113900661468